In [2]:

!pip install pandas numpy scikit-learn matplotlib plotly seaborn transformers datasets torch


Import Libraries

In [3]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import json, uuid, time


In [1]:
import nbformat

path = "C:\\Users\\arunr\\Downloads\\UC-13341-1 (2).ipynb"
nb = nbformat.read(path, as_version=4)

widgets = nb["metadata"].get("widgets", {})
for k, v in widgets.items():
    if isinstance(v, dict) and "state" not in v:
        v["state"] = {}

nb["metadata"]["widgets"] = widgets
nbformat.write(nb, path)


 Load Dataset

In [31]:

from google.colab import files
uploaded = files.upload()

df = pd.read_csv("validation.csv")
df.head(3)


Saving validation.csv to validation.csv


,dialog,act,emotion
0,"['Good morning , sir . Is there a bank near he...",[2 1 3 2 1 2 1],[0 0 0 0 0 0 0]
1,['Good afternoon . This is Michelle Li speakin...,[2 1 1 1 1 2 3 2 3 4],[0 0 0 0 0 0 0 0 0 0]
2,['What qualifications should a reporter have ?...,[2 1 2 1],[0 0 0 0]


Preprocess: Flatten Dialogs into Utterances

In [5]:

def parse_list_str(s):
    return s.strip("[]").replace("\n","").split()

utterances = []
for i, row in df.iterrows():
    dialog = row['dialog']
    acts = row['act']
    emotions = row['emotion']

    turns = [t.strip(" '\"") for t in dialog.split("\n") if len(t.strip())>0]
    act_labels = parse_list_str(acts)
    emo_labels = parse_list_str(emotions)

    for j, utt in enumerate(turns):
        if j < len(act_labels) and j < len(emo_labels):
            utterances.append({
                "text": utt,
                "act": act_labels[j],
                "emotion": emo_labels[j]
            })

utter_df = pd.DataFrame(utterances)
print("Utterances:", utter_df.shape)
utter_df.head(10)


Utterances: (7093, 3)


,text,act,emotion
0,"['Good morning , sir . Is there a bank near he...",2,0
1,There is one . 5 blocks away from here ?,1,0
2,"Well , that's too far.Can you change some mone...",3,0
3,"Surely , of course . What kind of currency hav...",2,0
4,How much would you like to change ? ' ' 1000 Y...,1,0
5,['Good afternoon . This is Michelle Li speakin...,2,0
6,"This is Mr Meng speaking , Michelle .",1,0
7,"Oh , hello ! Sorry about that . I'm just calli...",1,0
8,That was quick ! I wasn't expecting it until l...,1,0
9,"Yes , our application procedures have speeded ...",1,0


 Train-Test Split

In [6]:

train, test = train_test_split(utter_df, stratify=utter_df['act'], test_size=0.2, random_state=42)

X_train, y_train_act, y_train_emo = train['text'], train['act'], train['emotion']
X_test, y_test_act, y_test_emo = test['text'], test['act'], test['emotion']


Baseline NLP: TF-IDF + Logistic Regression (for ACT)

In [7]:

tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)

model_act = LogisticRegression(max_iter=200)
model_act.fit(X_train_vec, y_train_act)
y_pred_act = model_act.predict(X_test_vec)

print("=== Dialog Act Classification ===")
print(classification_report(y_test_act, y_pred_act))


=== Dialog Act Classification ===
              precision    recall  f1-score   support

           1       0.53      0.73      0.61       526
           2       0.62      0.59      0.61       425
           3       0.51      0.45      0.48       323
           4       0.28      0.03      0.06       145

    accuracy                           0.55      1419
   macro avg       0.49      0.45      0.44      1419
weighted avg       0.53      0.55      0.52      1419



Baseline NLP: TF-IDF + Logistic Regression (for EMOTION)

In [8]:

model_emo = LogisticRegression(max_iter=200)
model_emo.fit(X_train_vec, y_train_emo)
y_pred_emo = model_emo.predict(X_test_vec)

print("=== Emotion Classification ===")
print(classification_report(y_test_emo, y_pred_emo))


=== Emotion Classification ===
              precision    recall  f1-score   support

           0       0.91      1.00      0.95      1286
           1       0.00      0.00      0.00        14
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         2
           4       0.75      0.07      0.12        90
           5       0.00      0.00      0.00        10
           6       0.00      0.00      0.00        16

    accuracy                           0.91      1419
   macro avg       0.24      0.15      0.15      1419
weighted avg       0.87      0.91      0.87      1419



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Confusion Matrix (Intent)

In [9]:

cm = confusion_matrix(y_test_act, y_pred_act, labels=model_act.classes_)
fig = px.imshow(cm,
                x=model_act.classes_,
                y=model_act.classes_,
                text_auto=True,
                title="Confusion Matrix - Dialog Acts")
fig.show()


In [18]:
!pip install --upgrade transformers datasets accelerate -q


Advanced NLP: DistilBERT for Act Classification

In [22]:

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import numpy as np
from sklearn.metrics import accuracy_score, f1_score


train_hf = Dataset.from_pandas(train[['text','act']])
test_hf = Dataset.from_pandas(test[['text','act']])


label2id = {l:i for i,l in enumerate(utter_df['act'].unique())}
id2label = {v:k for k,v in label2id.items()}


train_hf = train_hf.map(lambda e: {"labels": label2id[e['act']]})
test_hf = test_hf.map(lambda e: {"labels": label2id[e['act']]})


tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(batch):
    return tokenizer(batch['text'], padding="max_length", truncation=True)

train_hf = train_hf.map(tokenize, batched=True)
test_hf = test_hf.map(tokenize, batched=True)

train_hf.set_format("torch", columns=["input_ids","attention_mask","labels"])
test_hf.set_format("torch", columns=["input_ids","attention_mask","labels"])


model_bert_act = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)




Map:   0%|          | 0/5674 [00:00<?, ? examples/s]

Map:   0%|          | 0/1419 [00:00<?, ? examples/s]

Map:   0%|          | 0/5674 [00:00<?, ? examples/s]

Map:   0%|          | 0/1419 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Intent Distribution (Ground Truth vs Predicted)

In [23]:

fig_true = px.histogram(utter_df, x="act", title="Ground Truth Intent Distribution")
fig_true.show()

y_pred_act = model_act.predict(tfidf.transform(X_test))
pred_df = pd.DataFrame({"predicted": y_pred_act})
fig_pred = px.histogram(pred_df, x="predicted", title="Predicted Intent Distribution (Baseline)")
fig_pred.show()


Emotion Distribution

In [24]:
fig_emo = px.histogram(utter_df, x="emotion", title="Ground Truth Emotion Distribution")
fig_emo.show()


Intent vs Emotion Heatmap

In [25]:
heatmap_data = utter_df.groupby(["act","emotion"]).size().reset_index(name="count")
heatmap_pivot = heatmap_data.pivot(index="act", columns="emotion", values="count").fillna(0)

fig = px.imshow(
    heatmap_pivot.values,
    x=heatmap_pivot.columns,
    y=heatmap_pivot.index,
    color_continuous_scale="Blues",
    title="Intent × Emotion Heatmap"
)
fig.show()


Confidence Histogram (Intent Model)

In [26]:
y_probs = model_act.predict_proba(tfidf.transform(X_test))
confidences = y_probs.max(axis=1)

fig = px.histogram(
    confidences, nbins=20,
    title="Intent Classification Confidence Distribution"
)
fig.show()


User Journey Sankey Diagram

In [27]:
utter_df['next_act'] = utter_df['act'].shift(-1)
trans = utter_df.dropna(subset=['next_act']).groupby(['act','next_act']).size().reset_index(name='count')

labels = list(set(trans['act']).union(set(trans['next_act'])))
label2id = {l:i for i,l in enumerate(labels)}

fig = go.Figure(go.Sankey(
    node=dict(label=labels, pad=20, thickness=20),
    link=dict(
        source=trans['act'].map(label2id),
        target=trans['next_act'].map(label2id),
        value=trans['count']
    )
))
fig.update_layout(title_text="User Journey - Intent Transitions", font_size=10)
fig.show()


Confusion Matrix for Emotions

In [28]:
from sklearn.metrics import confusion_matrix


y_pred_emo = model_emo.predict(tfidf.transform(X_test))


cm_emo = confusion_matrix(y_test_emo, y_pred_emo, labels=model_emo.classes_)

fig = px.imshow(
    cm_emo,
    x=model_emo.classes_,
    y=model_emo.classes_,
    text_auto=True,
    title="Confusion Matrix - Emotion Classification"
)
fig.show()
